In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
# import scipy.interpolate as interp
import scipy as sc
from numpy import pi, cos, sin, log, exp
import numpy as np
from datetime import datetime, timezone

%load_ext autoreload
%autoreload 2
import os
import sys
import time
if not '..' in sys.path:
    sys.path.append('..')


In [2]:
datetime.utcfromtimestamp(1684065764.0)

datetime.datetime(2023, 5, 14, 12, 2, 44)

In [3]:
datetime.utcfromtimestamp(1684072964.0)

datetime.datetime(2023, 5, 14, 14, 2, 44)

In [9]:
array1 = [47906.42060995102, 46995.42060995102, 19167.67854895592, 2201.768243980408]
array2 = [47906.42060995102, 46995.42060995102, 19167.67854895592, 2201.768243980408, 57482.57939004898]
print(np.linalg.norm(array1))
print(np.linalg.norm(array2))
print('\n')

array1 = [47906.42060995102, 46995.42060995102, 19167.67854895592, 2201.768243980408, 1000000]
array2 = [47906.42060995102, 46995.42060995102, 19167.67854895592, 2201.768243980408, 57482.57939004898, 1000000]
print(np.linalg.norm(array1))
print(np.linalg.norm(array2))

69827.23235471999
90443.84617901506


1002434.9566821381
1004081.7144593628


# Расчёт полосы съёмки

In [2]:
sector_coverage = 70*pi/180 # deg
depth = 9 # m
sv = 1475 # m/s

SR = depth/cos(sector_coverage)
print(SR)

swath_length = np.sqrt(SR**2 - depth**2)
print(swath_length)

26.31423960146778
24.727296775091595


# Parasound P-70 Metainfo files Parsing

In [ ]:
# Parasound P-70 Metainfo files Parsing
# meta_paths = find_ext('.inf')
# desired_path = meta_paths[1]

# output_file = os.path.splitext(desired_path)[0] + '_delays' + '.txt'
# print(output_file)

# header = ['Datetime', 'Delay[m]']
# delays_list = []
# datetime_list = []
# for path in meta_paths:
#     output_file = os.path.splitext(path)[0] + '_delays' + '.txt'
    
#     with open(path) as f:
#         content = f.read().splitlines()

#         for line in content:
#             line_content = line.split(',')

#             if line_content[1] == header[1]:
#                 datetime_list.append(line_content[0])
#                 delays_list.append(line_content[2])

#     with open(output_file, 'w') as f:
#         f.write(header[0] + ' ' + header[1] + '\n')

#         for num, line in enumerate(datetime_list):
#             f.write(datetime_list[num] + ' ' + delays_list[num] + '\n')

        
        

In [ ]:
# Parasound P-70 Metainfo files Parsing
# meta_paths = find_ext('.inf')
# desired_path = meta_paths[1]

# path = os.path.splitext(desired_path)[0] + '_delays' + '.txt'
# print(output_file)

# header = ['Datetime', 'Delay[m]']
# delays_list = []
# datetime_list = []

# output_file = os.path.splitext(path)[0] + '_delays' + '.txt'
# with open(path) as f:
#     content = f.read().splitlines()

#     for line in content:
#         line_content = line.split(',')

#         if line_content[1] == header[1]:
#             datetime_list.append(line_content[0])
#             delays_list.append(line_content[2])
# with open(output_file, 'w') as f:
#     f.write(header[0] + ' ' + header[1] + '\n')

#     for num, line in enumerate(datetime_list):
#         f.write(datetime_list[num] + ' ' + delays_list[num] + '\n')

# MiniSVP format parser.

Скрипт для выгрузки из .TXT Valeport MiniSVP метаданных о времени записи файла, его имени и серийном номере зонда.

Это необходимо для формирования таблицы SVP зондирований

In [16]:
import os
from mycode.find_files_with_extension import find_file_with_extension

def write_miniSVP_metadata(SVP_path, SVP_list_path):
    with open(SVP_path, 'r') as f:
        file_content = f.read().splitlines()

    date = file_content[0].split(' ')[1].split('/')
    time = file_content[0].split(' ')[2].split(':')

    datetime = date[2] + date[1] + date[0] + '-' + time[0] + time[1] + '00.000000'
    serial_num = file_content[2].split(' ')[2]
    filename = os.path.split(SVP_path)[-1].split('.')[0]

    if os.path.exists(SVP_list_path):
        with open(SVP_list_path, 'a') as f:
            f.write(f"{filename}\t{serial_num}\t{datetime}\n")
    else:
        with open(SVP_list_path, 'w') as f:
            f.write('Fname\tS/N\tDatetime\n')
            f.write(f"{filename}\t{serial_num}\t{datetime}\n")


In [18]:
SVP_paths = find_file_with_extension(extension='.TXT', path=r'Z:\ABP_50_TTR-21\af_SVPData\MiniSVP raw')

for path in SVP_paths:
    write_miniSVP_metadata(path, r'Z:\ABP_50_TTR-21\af_SVPData\MiniSVP raw\SVP_metadata_file.txt')

Searching *.TXT files in directory:Z:\ABP_50_TTR-21\af_SVPData\MiniSVP raw


# Преобразование csv Sound Speed Manager в .svp PDS2000

Скрипт для конвертации обработанных в SoundSpeedManager .TXT данных Valeport MiniSVP в формат .svp для проекта PDS2000

Для данного скрипта необходим журнал SVP зондирований, с шапкой: ['NumO', 'Num', 'S/N', 'FileName', 'Date', 'TimeUTC', 'DateTimeUTCOpCPN', 'DateTimeUTCProbe', 'Station', 'Location', 'latN', 'lonE']

S/N - серийный номер зонда; FileName - Имя файла .TXT; DateTimeUTCOpCPN -  время станции зондирования в UTC (не время из шапки файла .TXT; Station - название/номер станции

In [3]:
import os
from mycode.find_files_with_extension import find_file_with_extension

def read_my_svp_logbook(path):
    sn_list = []
    fn_list = []
    dt_list = []
    st_list = []

    with open(path) as f:
        file_content = f.read().splitlines()
        header = 1
        for line in file_content:
            if header == 1:
                pass
            else:
                line_content = line.split('\t')
                # header: ['NumO', 'Num', 'S/N', 'FileName', 'Date', 'TimeUTC', 'DateTimeUTCOpCPN', 'DateTimeUTCProbe', 'Station', 'Location', 'latN', 'lonE']
                sn_list.append(line_content[2])
                fn_list.append(line_content[3])
                dt_list.append(line_content[6])
                st_list.append(line_content[8])
            header = 0
        return sn_list, fn_list, dt_list, st_list

def modify_output_file_name(serial_num, logpath, output_path, filetype='SSMminiSVP'):
    # Read logbook
    sn_list, fn_list, dt_list, st_list = read_my_svp_logbook(logpath)
    # Get the file name
    out_folder_path, out_fname_ext2_ext1 = os.path.split(output_path)
    out_fname_ext2, out_ext1 = os.path.splitext(out_fname_ext2_ext1)
    out_fname, ext2 = os.path.splitext(out_fname_ext2)

    if filetype == 'SSMminiSVP':
        fname = out_fname
    elif filetype == 'Idronaut' or filetype == 'CTD90':
        fname = out_fname_ext2
    
    # Find the file in the logbook
    for lnum, fn in enumerate(fn_list):
        if fn == fname:
            if sn_list[lnum] == str(serial_num):
                # Format: yyyy-mm-dd-HHMM
                mod_time = f'{dt_list[lnum][0:4]}-{dt_list[lnum][4:6]}-{dt_list[lnum][6:8]}-{dt_list[lnum][8:]}'
                mod_out_fname = f'{mod_time}_{st_list[lnum]}_{sn_list[lnum]}'
                # modify output path
                mod_fname_ext = mod_out_fname + '.svp'
                mod_out_path = os.path.join(out_folder_path, mod_fname_ext)
                
                if mod_out_path is None:
                    raise RuntimeError('Modified path is None. Something wrong with the code or the data')
                return mod_out_path

def convert_SMMcsv_to_PDSsvp(SSMcsv_path, PDSsvp_path):
    # Step 1. Read Sound Speed Manager .csv file
    pdepth = []
    pSV = []
    ptemp = []
    psal = []

    with open(SSMcsv_path, 'r') as f:
        # field order: depth, sound speed, salinity, temperature
        line_start = 5  # previous 5 lines are parts of a header
        filecontent = f.read().splitlines()[line_start:]
        for line in filecontent:
            line_content = line.split(',')
            pdepth.append(float(line_content[0]))
            pSV.append(float(line_content[1]))
            ptemp.append(float(line_content[3]))
            psal.append(float(line_content[2]))

    # Step 2. Create a new PDS2000 .svp and update it by SSM .csv values
    pheader = '[POINTS]\n'
    pnum_end = len(pdepth)
    offset_header = '\n[OFFSET]\n'
    sv_offset_name = 'SVOffset'
    sv_offset = '0'
    sv_offset_line = '{:} = {:}\n\n'.format(sv_offset_name,
                                        sv_offset)

    # field order: depth, sound velocity, temperature, salinity
    with open(PDSsvp_path, 'w') as f:
        f.write(pheader)
        for pnum in range(pnum_end):
            # field order: depth, sound velocity, temperature, salinity
            point = 'Point({:d}) = {:f},{:f},{:f},{:f}\n'.format(pnum+1,
                                                                 pdepth[pnum]*-1, 
                                                                 pSV[pnum], 
                                                                 ptemp[pnum], 
                                                                 psal[pnum])
            f.write(point)
        f.write(offset_header)
        f.write(sv_offset_line)

############################

serial_number_list - list с серийными номерами зондов

data_folder_list1 - list с путями до папок с .csv данными (обработанными в SSM данными SVP зондов)

my_svp_log_path - путь до журнала SVP зондирований

In [5]:
# serial_number_list = [43946, 75748, 75749]
serial_number_list = [43946, 75748]
data_folder_list1 = ['D:\\MyExpeditions\\Cr_20220601_ABP_049\\data_CTD_SVP\\Data\\SoundSpeedManager\\43946_proc\\To process',
                    'D:\\MyExpeditions\\Cr_20220601_ABP_049\\data_CTD_SVP\\Data\\SoundSpeedManager\\75748_proc\\To process']

my_svp_log_path = 'D:\\MyExpeditions\\Cr_20220601_ABP_049\\data_CTD_SVP\\ABP049_SVP_ProcessingTable.txt'


for sn, df in zip(serial_number_list, data_folder_list1):
    SSM_csv_paths = find_file_with_extension('.csv', df)
    
    for SSM_csv_path in SSM_csv_paths:
        PDS_svp_path = modify_output_file_name(sn, my_svp_log_path, SSM_csv_path)
        convert_SMMcsv_to_PDSsvp(SSM_csv_path, PDS_svp_path)
        

Searching *.csv files in directory:D:\MyExpeditions\Cr_20220601_ABP_049\data_CTD_SVP\Data\SoundSpeedManager\43946_proc\To process
Searching *.csv files in directory:D:\MyExpeditions\Cr_20220601_ABP_049\data_CTD_SVP\Data\SoundSpeedManager\75748_proc\To process


# Преобразование .csv Idronaut от Андрея Коржа в .svp PDS2000

In [ ]:
def convert_idronautcsv_to_PDSsvp(idronautcsv_path, PDSsvp_path):
        # Step 1. Read Sound Speed Manager .csv file
    pdepth = []
    pSV = []
    ptemp = []
    psal = []

    with open(idronautcsv_path, 'r') as f:
        # field order: depth, sound speed, salinity, temperature
        line_start = 1  # first line is a header
        filecontent = f.read().splitlines()[line_start:]
        for line in filecontent:
            line_content = line.split(',')
            pdepth.append(float(line_content[9]))
            pSV.append(float(line_content[10]))
            ptemp.append(float(line_content[2]))
            psal.append(float(line_content[4]))

    # Step 2. Create a new PDS2000 .svp and update it by SSM .csv values
    pheader = '[POINTS]\n'
    pnum_end = len(pdepth)
    offset_header = '\n[OFFSET]\n'
    sv_offset_name = 'SVOffset'
    sv_offset = '0'
    sv_offset_line = '{:} = {:}\n\n'.format(sv_offset_name,
                                        sv_offset)

    # field order: depth, sound velocity, temperature, salinity
    with open(PDSsvp_path, 'w') as f:
        f.write(pheader)
        for pnum in range(pnum_end):
            # field order: depth, sound velocity, temperature, salinity
            point = 'Point({:d}) = {:f},{:f},{:f},{:f}\n'.format(pnum+1,
                                                                 pdepth[pnum]*-1, 
                                                                 pSV[pnum], 
                                                                 ptemp[pnum], 
                                                                 psal[pnum])
            f.write(point)
        f.write(offset_header)
        f.write(sv_offset_line)

############################

In [ ]:
logpath = 'F:\\ABP_049\\Станции, логбуки и треки\\ABP049_Idronaut_Log.txt'
datapath = 'F:\\ABP_049\\Станции, логбуки и треки\\CTD_Koleso\\CTD_Idronaut_OS316#494'

sn = 'Idronaut'
idronaut_csv_paths = find_file_with_extension('.csv', datapath)

for idronaut_csv_path in idronaut_csv_paths:
    PDS_svp_path = modify_output_file_name(sn, logpath, idronaut_csv_path, filetype='Idronaut')
    convert_idronautcsv_to_PDSsvp(idronaut_csv_path, PDS_svp_path)


# Преобразование .csv CTD90 от Виктора Кречика в PDS2000 .svp

In [4]:
def convert_ctd90csv_to_PDSsvp(idronautcsv_path, PDSsvp_path):
        # Step 1. Read Sound Speed Manager .csv file
    pdepth = []
    pSV = []
    ptemp = []
    psal = []

    with open(idronautcsv_path, 'r') as f:
        # field order: depth, sound speed, salinity, temperature
        line_start = 35  # Previous lines are a header
        filecontent = f.read().splitlines()[line_start:]
        for line in filecontent:
            line_content = line.split(',')
            depth = float(line_content[2])
            sv = float(line_content[8].split(';')[0])
            temp = float(line_content[3])
            sal = float(line_content[4])
            if depth <= 0:
                pass
            elif sal < 1:
                pass
            elif len(pdepth) > 1:
                if len(pdepth) == 1:
                    pass
                elif pdepth[-1] >= depth and pdepth[-2] >= depth:
                    pass
                else:
                    pdepth.append(depth)
                    pSV.append(sv)
                    ptemp.append(temp)
                    psal.append(sal)
            else:
                pdepth.append(depth)
                pSV.append(sv)
                ptemp.append(temp)
                psal.append(sal)

    # Step 2. Create a new PDS2000 .svp and update it by SSM .csv values
    pheader = '[POINTS]\n'
    pnum_end = len(pdepth)
    offset_header = '\n[OFFSET]\n'
    sv_offset_name = 'SVOffset'
    sv_offset = '0'
    sv_offset_line = '{:} = {:}\n\n'.format(sv_offset_name,
                                        sv_offset)

    # field order: depth, sound velocity, temperature, salinity
    with open(PDSsvp_path, 'w') as f:
        f.write(pheader)
        for pnum in range(pnum_end):
            # field order: depth, sound velocity, temperature, salinity
            point = 'Point({:d}) = {:f},{:f},{:f},{:f}\n'.format(pnum+1,
                                                                 pdepth[pnum]*-1, 
                                                                 pSV[pnum], 
                                                                 ptemp[pnum], 
                                                                 psal[pnum])
            f.write(point)
        f.write(offset_header)
        f.write(sv_offset_line)

############################

In [ ]:
logpath = 'F:\\ABP_049\\Станции, логбуки и треки\\ABP049_CTD90_Log.txt'
datapath = 'F:\\ABP_049\\Станции, логбуки и треки\\CTD_Krechik\\CTD90'

sn = 'CTD90'
ctd90_csv_paths = find_file_with_extension('.CSV', datapath)

for ctd90_csv_path in ctd90_csv_paths:
    PDS_svp_path = modify_output_file_name(sn, logpath, ctd90_csv_path, filetype='CTD90')
    convert_ctd90csv_to_PDSsvp(ctd90_csv_path, PDS_svp_path)

# Скрипт для разделения .csv PDS трека записей на множество .csv, индивидуальных для каждого файла, а затем объединение их в .csv треков файлсетов

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate as interp
import scipy
from numpy import pi, cos, sin, log, exp
import numpy as np
from datetime import datetime, timezone

from mycode.find_files_with_extension import find_file_with_extension
from mbesmanager.pds_fileset_class import PDSFileset
from mbesmanager.pds_filetrack_class import PDSFileTrack, read_filetrack_csv


In [3]:
def create_pdsfile_track_csv(ftr, storage_path):
    ftr_name = ftr.fname
    path = os.path.join(storage_path, ftr_name+'.csv')
    
    with open(path, 'w') as f:
        header = f'Project,Fname,Time,Projection,X,Y,SSV,Depth,Heading\n'
        f.write(header)
        for index, time in enumerate(ftr.tr_time):
            f.write(f'{ftr.project},{ftr.fname},{datetime.utcfromtimestamp(time).isoformat()},{ftr.projection},{ftr.tr_x[index]},{ftr.tr_y[index]},{ftr.tr_sv[index]},{ftr.tr_depth[index]},{ftr.tr_heading[index]}\n')

def create_fset_path_csv(fs_obj, ft_obj_list, storage_path):
    fileset_name = os.path.splitext(fs_obj.fileset_name)[0]
    path_st = os.path.join(storage_path, fileset_name + '.csv')

    with open(path_st, 'w') as f:
        header = f'Project,FSname,Fname,Time,Projection,X,Y,SSV,Depth,Heading\n'
        f.write(header)
        for ft_obj in ft_obj_list:
            if ft_obj.fname + ft_obj.fext in fs_obj.linked_pds_files_names:
                for index, time in enumerate(ft_obj.tr_time):
                    f.write(f'{ft_obj.project},{fileset_name},{ft_obj.fname},{datetime.utcfromtimestamp(time).isoformat()},{ft_obj.projection},{ft_obj.tr_x[index]},{ft_obj.tr_y[index]},{ft_obj.tr_sv[index]},{ft_obj.tr_depth[index]},{ft_obj.tr_heading[index]}\n')
                    
def create_fsets_path_csv(fs_obj_list, ft_obj_list, output_csv_file_path):
    with open(output_csv_file_path, 'w') as f:
        header = f'Project,FSname,Fname,Time,Projection,X,Y,SSV,Depth,Heading\n'
        f.write(header)
        for fs_obj in fs_obj_list:
            fs_name = os.path.splitext(fs_obj.fileset_name)[0]
            for ft_obj in ft_obj_list:
                if ft_obj.fname + ft_obj.fext in fs_obj.linked_pds_files_names:
                    for index, time in enumerate(ft_obj.tr_time):
                        f.write(f'{ft_obj.project},{fs_name},{ft_obj.fname},{datetime.utcfromtimestamp(time).isoformat()},{ft_obj.projection},{ft_obj.tr_x[index]},{ft_obj.tr_y[index]},{ft_obj.tr_sv[index]},{ft_obj.tr_depth[index]},{ft_obj.tr_heading[index]}\n')

def create_fsets_path_csv_ts_te(fs_obj_list, ft_obj_list, output_csv_file_path, error_list=os.path.join(os.getcwd(), 'create_fsets_path_csv_ts_te_errors.txt')):
    with open(output_csv_file_path, 'w') as f:
        header = f'Project,FSname,Fname,Time,StOrEn,Projection,X,Y,SSV,Depth,Heading\n'
        f.write(header)
        for fs_obj in fs_obj_list:
            fs_name = os.path.splitext(fs_obj.fileset_name)[0]
            for ft_obj in ft_obj_list:
                if ft_obj.fname + ft_obj.fext in fs_obj.linked_pds_files_names:
                    index_srt = 0
                    index_end = -1
                    try:
                        ft_obj.tr_time[0]
                    except:
                        print(f"File {ft_obj.fname} in fileset {fs_name} doesn't have start nor end values")
                        err_file = open(error_list, 'a')
                        err_file.write(f'{datetime.utcnow().isoformat()} {ft_obj.fname}\n')
                        err_file.close()
                                
                    else:
                        f.write(f'{ft_obj.project},{fs_name},{ft_obj.fname},{datetime.utcfromtimestamp(ft_obj.tr_time[0]).isoformat()},Start,{ft_obj.projection},{ft_obj.tr_x[0]},{ft_obj.tr_y[0]},{ft_obj.tr_sv[0]},{ft_obj.tr_depth[0]},{ft_obj.tr_heading[0]}\n')
                        f.write(f'{ft_obj.project},{fs_name},{ft_obj.fname},{datetime.utcfromtimestamp(ft_obj.tr_time[-1]).isoformat()},End,{ft_obj.projection},{ft_obj.tr_x[-1]},{ft_obj.tr_y[-1]},{ft_obj.tr_sv[-1]},{ft_obj.tr_depth[-1]},{ft_obj.tr_heading[-1]}\n')
                    
def fileset_load_tr(fileset_paths_from):
    filesets = list()
    size = 0
    file_number = 0
    fileset_number = 0
    for path in fileset_paths_from:
        fileset = PDSFileset()
    
        # path - is a full path to the *.sub fileset
        split_path = path.split(sep='\\')
        fileset.fileset_name = split_path[-1]
        fileset.fileset_path = path

        with open(path) as f:
            file_read = f.readlines()
            for idx, line in enumerate(file_read):
                line_content = line.splitlines()[0]
                
                # Declare the header
                if idx == 0:
                    fileset.header = line_content
                # Cut the end
                elif (line_content.rstrip('\n')).rstrip(' ') == '':
                    pass
                # The content
                else:
                    pds_project_path = os.path.split(path)[0]
                    split_line = line_content.split(sep=fileset.equal_symbol)
                    fileset.file_num.append(split_line[0])
                    split_path = split_line[1].split(sep='\\')
                    fileset.relative_path_dir.append(split_path[0])
                    fileset.relative_path_file.append(split_path[1])

                    fileset.linked_pds_files_names.append(split_path[1] + '.pds')
                    fileset.linked_pds_files_paths.append(os.path.join(pds_project_path,
                                                                    fileset.relative_path_dir[-1],
                                                                    fileset.linked_pds_files_names[-1]))

                    fileset.linked_gpt_files_names.append(split_path[1] + '.gpt')
                    fileset.linked_gpt_files_paths.append(os.path.join(pds_project_path,
                                                                    fileset.relative_path_dir[-1],
                                                                    fileset.linked_gpt_files_names[-1]))
        filesets.append(fileset)
    return filesets



In [4]:
tracktable_path = 'Z:\\ABP_50_TTR-21\\zzzz_IvanDudkovFolder\\TrackProcessing\\track_spreadsheet\\VesselTrack_CLP2_v2.csv'
filesets_path = 'Z:\\ABP_50_TTR-21\\zzzz_IvanDudkovFolder\\TrackProcessing\\filesets'
######
output_file_path = 'Z:\\ABP_50_TTR-21\\zzzz_IvanDudkovFolder\\TrackProcessing\\track_spreadsheet\\VesselTrack_CLP2_proc_v2.csv'
output_srt_end_file_path = 'Z:\\ABP_50_TTR-21\\zzzz_IvanDudkovFolder\\TrackProcessing\\track_spreadsheet\\VesselTrack_CLP2_StEnd_v2.csv'
error_list = 'Z:\\ABP_50_TTR-21\\zzzz_IvanDudkovFolder\\TrackProcessing\\track_spreadsheet\\VesselTrack_CLP2_ProcErrList_v2.csv'

# tr_csv_storage_path = 'C:\\Workspace\\Projects\\Pr_MBESManager\\PDS2000_TrackProcessing\\pds_csv_tracks'
# fs_csv_storage_path = 'C:\\Workspace\\Projects\\Pr_MBESManager\\PDS2000_TrackProcessing\\filesets_csv_tracks'

fs_obj_paths = find_file_with_extension('.sub', filesets_path)
fs_obj_list = fileset_load_tr(fs_obj_paths)
ft_obj_list = read_filetrack_csv(tracktable_path, 'LambConfConic2Par')

# Create one CSV file for all filesets and files. Only starts and ends of files:
create_fsets_path_csv_ts_te(fs_obj_list, ft_obj_list, output_srt_end_file_path, error_list=error_list)

# Create one CSV file for all filesets and files:
create_fsets_path_csv(fs_obj_list, ft_obj_list, output_file_path)


# Create CSV file for each .pds track:
# for ft in ft_obj_list:
#     create_pdsfile_track_csv(ft, tr_csv_storage_path)

# Create CSV file for each .sub fileset:
# for fs in fs_obj_list:
#     create_fset_path_csv(fs, ft_obj_list, fs_csv_storage_path)


Searching *.sub files in directory:Z:\ABP_50_TTR-21\zzzz_IvanDudkovFolder\TrackProcessing\filesets
error at line 302626. depth: 
error at line 302627. depth: 
error at line 302628. depth: 
error at line 302633. depth: 
error at line 302634. depth: 
error at line 302635. depth: 
error at line 302636. depth: 
error at line 302637. depth: 
error at line 302638. depth: 
error at line 302639. depth: 
error at line 302640. depth: 
error at line 302641. depth: 
error at line 302642. depth: 
error at line 302643. depth: 
error at line 302644. depth: 
error at line 302645. depth: 
error at line 302646. depth: 
error at line 302647. depth: 
error at line 302648. depth: 
error at line 302649. depth: 
error at line 302650. depth: 
error at line 302651. depth: 
error at line 302652. depth: 
error at line 302653. depth: 
error at line 302654. depth: 
error at line 302655. depth: 
error at line 302656. depth: 
error at line 302657. depth: 
error at line 302658. depth: 
error at line 302659. depth: 
e